In [ ]:
import pandas as pd
import plotly.express as px
import geopandas as gpd

In [ ]:
lsoas = gpd.read_file("Lower_layer_Super_Output_Areas_December_2021_Boundaries_EW_BSC_V4_-4299016806856585929.geojson")


In [ ]:
lsoas.boundary.plot(color = 'k', linewidth = 0.1)

In [ ]:
print(lsoas.head())

In [ ]:
lsoas.geometry.iloc[1]

In [ ]:
consumption_by_lsoa = pd.read_excel("final_data.xlsx")
consumption_by_lsoa.head(3)


In [ ]:
filtered_data = consumption_by_lsoa[['LSOA code','Electricity Total Consumption (kWh)','geometry','longitude','latitude']]
data_to_plot = filtered_data.rename(columns={"Electricity Total Consumption (kWh)": "consumption"})
data_to_plot.head(3)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler() 
scaled_consumption = scaler.fit_transform(data_to_plot[['consumption']]) 
scaled_consumption_df = pd.DataFrame(scaled_consumption[:, 0])
scaled_consumption_df.rename(columns={0: "scaled_consumption"}, inplace=True)
scaled_consumption_df.head(3)

In [ ]:
# Add scaled consumption to dataframe

data_to_plot['scaled_consumption'] = scaled_consumption_df
plot_data = data_to_plot[['LSOA code','scaled_consumption']]
plot_data.head(3)

In [ ]:
lsoas.sort_values('LSOA21CD')


In [ ]:
data = lsoas.merge(plot_data, left_on='LSOA21CD',right_on='LSOA code')
data.head(3)

In [ ]:
fig = px.choropleth_map(data, 
                         geojson = data.geometry, 
                         locations=data.index,
                         color=data.scaled_consumption,
                         color_continuous_scale="twilight",
                         map_style="carto-positron",
                         zoom = 4.5, center = {"lat":51.5072,"lon":0.1276})
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
